<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [ ]:
#@title # Install Conda (requires runtime restart) { vertical-output: true, display-mode: "form" }

!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:18
🔁 Restarting kernel...


In [ ]:
#@title # Install Nerfstudio and Dependencies (~10 min) { vertical-output: true, display-mode: "form" }

%cd /content/
!pip install --upgrade pip
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# Installing TinyCuda
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1q8fuc-Mqiev5GTBTRA5UPgCaQDzuqKqj" 
!pip install tinycudann-1.6-cp37-cp37m-linux_x86_64.whl

# Installing COLMAP
%cd /content/
!conda install -c conda-forge colmap

# Install nerfstudio
%cd /content/
!pip install nerfstudio

In [ ]:
#@markdown <h1> Downloading and Processing Data</h1>
#@markdown <h3>Pick the preset scene or upload your own images/video</h3>
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDDBC poster' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene not in ['upload your images', 'upload your own video', 'upload your own Record3D data']:
    %cd /content/
    !ns-download-data --dataset=nerfstudio --capture=$scene
elif scene == 'upload your own Record3D data':
    display(HTML('<h3>Zip your Record3D folder, and upload.</h3>'))
    display(HTML('<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'))
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
else:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on hires inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
print("Data Processing Succeeded!")

/content
Downloading...
From: https://drive.google.com/uc?id=1dmjWGXlJnUxwosN6MVooCDQe970PkD-1
To: /content/data/nerfstudio/poster.zip
100% 750M/750M [00:02<00:00, 295MB/s]
Data Processing Succeeded!


In [ ]:
#@title #COLMAP結果を保存/復帰{vertical-output:true,display-mode:"form"}

import shutil
from google.colab import drive
drive.mount('/gdrive') 

state = 'Continue' #@param ['First play', 'Continue']
if state == 'First play': # 処理結果をColabからGoogle Driveに保存する
    shutil.copytree( "/content/data/nerfstudio/custom_data",
                     "/gdrive/MyDrive/Colab Notebooks/custom_data")
else:                # 既存の処理結果をGoogle DriveからColabに復帰する
    scene = "custom_data"
    shutil.copytree( "/gdrive/MyDrive/Colab Notebooks/custom_data",
                     "/content/data/nerfstudio/custom_data")


#%ls /content/data/nerfstudio/custom_data


# 以前にも実行していたら
#%rm - rf "/gdrive/MyDrive/Colab Notebooks/custom_data"

#%ls "/gdrive/MyDrive/Colab Notebooks/custom_data"

Mounted at /gdrive


In [ ]:
#@title # Set up and Start Viewer

%cd /content

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("You may need to click Refresh Page after you start training!")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

/content
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 0.406s
https://viewer.nerf.studio/?websocket_url=wss://hot-rabbits-rescue-35-230-172-213.loca.lt
You may need to click Refresh Page after you start training!


In [ ]:
#@title # Start Training {vertical-output:true}

import os
from IPython.core.display import display, HTML

%cd /content
state = "First play" #@param ["First play","Continue"] {allow-input:true}
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    if state == 'First play':
        !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
    else: # Google Driveから、既存の処理結果をColabに復元して、学習実行
        shutil.copytree( "/gdrive/MyDrive/Colab Notebooks/outputs",
                         "/content/outputs")
        file_timestamp = '2022-11-14_122043' # 復帰させたい計算結果
        !ns-train nerfacto --viewer.websocket-port 7007 --trainer.load_dir outputs/data-nerfstudio-custom_data/nerfacto/{file_timestamp}/nerfstudio_models nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    display(HTML('<h3 style="color:red">カメラ情報がありません</h3>'))

#from google.colab import drive
#drive.mount('/gdrive', force_remount=True) 

In [ ]:
#@title # 学習開始 {vertical-output:true}

import os
from IPython.core.display import display, HTML

%cd /content
state = "First play" #@param ["First play","Continue"] {allow-input:true}
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    if state == 'First play':
        !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
    else: # Google Driveから、既存の処理結果をColabに復元して、学習実行
        #shutil.copytree( "/gdrive/MyDrive/Colab Notebooks/outputs",
        #                 "/content/outputs")
        file_timestamp = '2022-11-14_122043' # 復帰させたい計算結果
        !ns-train nerfacto --viewer.websocket-port 7007 --trainer.load_dir outputs/data-nerfstudio-custom_data/nerfacto/{file_timestamp}/nerfstudio_models nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4

        #%rm - rf "/content/outputs"
        # Google Driveから、既存の処理結果をColabに復元して、学習実行実行
        #shutil.copytree( "/gdrive/MyDrive/Colab Notebooks/outputs",
        #                 "/content/outputs")
else:
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

In [ ]:
#@title # 学習結果の保存 { vertical-output: true }
shutil.copytree( "/content/outputs",
                 "/gdrive/MyDrive/Colab Notebooks/outputs")

#!rm - rf "/gdrive/MyDrive/Colab Notebooks/outputs"

In [ ]:
!ls /content/outputs/data-nerfstudio-custom_data/nerfacto
!ls /content/data/nerfstudio/custom_data/
#print(scene)
#scene = "custom_data"

In [ ]:
# 動画の保存（ViewerのRENDERタブからRENDERボタンを押すと、このコマンドを実行するように言われる）
!ns-render --load-config outputs/data-nerfstudio-custom_data/nerfacto/2022-11-15_033259/config.yml --traj filename --camera-path-filename outputs/data-nerfstudio-custom_data/nerfacto/2022-11-15_033259/camera_path.json --output-path renders/render_output.mp4

In [ ]:
#@title # Start Training { vertical-output: true }

%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

In [ ]:
#@title # Render Video { vertical-output: true }
#@markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
#@markdown <h5>The rendered video should be at renders/output.mp4!</h5>
from google.colab import files

base_dir = "/content/outputs/data-nerfstudio-" + scene + "/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import display, HTML
display(HTML('<h3>Upload the camera path JSON.</h3>'))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render --load-config $config_filename --traj filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4